In [59]:
%pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge  # Example estimator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV,cross_validate
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

Note: you may need to restart the kernel to use updated packages.


##Download both the datasets and Preprocess
#Missing Data Found in Bmarketing| Added One Hot Encoding and padded missing data

In [69]:
bank_marketing = pd.read_csv('/workspaces/7641HW1/data/BankMarketingData.csv')
breast_cancer = fetch_ucirepo(id=17)
X2 = pd.DataFrame(breast_cancer.data.features, columns=breast_cancer.feature_names)
y2 = pd.DataFrame(breast_cancer.data.targets)
dataset_2 = pd.concat([X2, y2], axis=1)
scaler = StandardScaler()
num_var =  ['radius1', 'texture1', 'perimeter1', 'area1', 'smoothness1',
       'compactness1', 'concavity1', 'concave_points1', 'symmetry1',
       'fractal_dimension1', 'radius2', 'texture2', 'perimeter2', 'area2',
       'smoothness2', 'compactness2', 'concavity2', 'concave_points2',
       'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'perimeter3',
       'area3', 'smoothness3', 'compactness3', 'concavity3', 'concave_points3',
       'symmetry3', 'fractal_dimension3']
dataset_2[num_var] = scaler.fit_transform(dataset_2[num_var])
#print(y2.value_counts())
dataset_2['Diagnosis'] = dataset_2['Diagnosis'].replace({'M': 1, 'B': 0}).astype('category')

#bankmarketing preprocessing
bank_marketing['y'] = bank_marketing['y'].replace({'yes': 1, 'no': 0}).astype('category')
bank_marketing = bank_marketing.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x.replace('unknown', x.mode()[0]).fillna(x.mode()[0]))
# Last_Column = bank_marketing.iloc[:, -1]  # This selects the last column
# y1 = pd.DataFrame(Last_Column)
# Last_Column = bank_marketing.iloc[:, -1] 
# y1 = pd.DataFrame(Last_Column)
# y1['y'] = y1['y'].replace({'yes': 1, 'no': 0})
# all_except_last = bank_marketing.iloc[:, :-1]  # This selects all columns except the last one
# # Step 3: Create a new DataFrame with the selected columns
# X1 = pd.DataFrame(all_except_last)
# X1['previous_contact'] = X1['pdays'].apply(lambda x: False if x == 999 else True)
# categorical_cols = X1.select_dtypes(include=['object', 'category']).columns.tolist()
# numerical_cols = X1.select_dtypes(include=['number']).columns.tolist()
# X1[numerical_cols]=scaler.fit_transform(X1[numerical_cols])
# X1 = X1.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'biufc' 
#                      else x.replace('unknown', x.mode()[0]).fillna(x.mode()[0]))
# X1_encoded = pd.get_dummies(X1, categorical_cols)
# X1 = X1_encoded

In [70]:
bank_marketing

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45302,30,admin.,married,basic.6y,no,yes,yes,cellular,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,0
45303,39,admin.,married,high.school,no,yes,no,telephone,jul,fri,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.959,5228.1,0
45304,27,student,single,high.school,no,no,no,cellular,may,mon,...,2,999,1,failure,-1.8,92.893,-46.2,1.354,5099.1,0
45305,58,admin.,married,high.school,no,no,no,cellular,aug,fri,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.966,5228.1,0


##Helper Functions for 
 Loading the Data,
 Running the Models,
 Showcasing the Results

In [63]:
def loading_data():
    X1 = X1.values.astype(np.float32)
    y1 = y1.values.ravel().astype(np.int64)


In [62]:
a

array([[ 1.5337    ,  0.01101394, -0.56856984, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.6297495 , -0.4216501 , -0.56856984, ...,  0.        ,
         1.        ,  0.        ],
       [-0.29124174, -0.12419358, -0.56856984, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.2517374 , -0.75001127, -0.20522824, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.7257991 ,  1.0424541 , -0.56856984, ...,  0.        ,
         1.        ,  0.        ],
       [-0.5793904 , -0.32121027, -0.56856984, ...,  0.        ,
         1.        ,  0.        ]], dtype=float32)